# Data Manipulation with Pandas 

In [ ]:
import pandas as pd
pd.set_option('max_rows', 10)

## Types de données "catégoriques"

* En plus de valeurs numériques et String, Pandas offre un `dtype` spécial pour des données catégoriques

In [ ]:
c = pd.Categorical(['a', 'b', 'b', 'c', 'a', 'b', 'a', 'a', 'a', 'a'])

In [ ]:
c

In [ ]:
c.describe()

In [ ]:
c.codes

In [ ]:
c.categories

### Support aux Catégories dans les DataFrames

* Lorsqu'une donnée catégorique est présente dans un DataFrame, il est possible de l'accéder avec l'opérateur `cat`
* Ceci donne accès aux différentes fonctionnalités du type Categorical

In [ ]:
import numpy as np
dta = pd.DataFrame.from_dict({'factor': c,
                              'x': np.random.randn(10)})

In [ ]:
dta.head()

In [ ]:
dta.dtypes

In [ ]:
dta.factor.cat

In [ ]:
dta.factor.cat.categories

In [ ]:
dta.factor.describe()

### Exercice

* Lire le dataset NFS à nouveau. Convertir `fditemno` en tant que type Categorical. Afficher le résultat avec describe.

In [ ]:
# [Solution Here]

In [ ]:
%load solutions/load_nfs_categorical.py


## Travailler avec dates et timestamps

### Créer une plage de dates

In [ ]:
dates = pd.date_range("1/1/2015", periods=75, freq="D")
dates

In [ ]:
y = pd.Series(np.random.randn(75), index=dates)
y.head()

In [ ]:
y.reset_index().dtypes

### Support au type datetime dans DataFrames

* Le type `datetime` a un opérateur spécial `dt` associé


In [ ]:
dta = (y.reset_index(name='t').
       rename(columns={'index': 'y'}))

In [ ]:
dta.head()

In [ ]:
dta.dtypes

In [ ]:
dta.y.dt.freq

In [ ]:
dta.y.dt.day

### Recherche avec des Dates

* Il est possible de faire une recherche avec des string
* **Note** : Cette recherche est inclusive (le dernier élément fait partie du résultat)

In [ ]:
y.loc["2015-01-01":"2015-01-15"]

Il est aussi possible de faire une recherche avec une partie de l'information

In [ ]:
y["2015-01"]

* Nous pouvons aussi faire un **resample**, réduisant la fréquence des données selon un moyen d'agrégation donné
* Pour cela, utiliser l'objet `DateTeimIndexResampler`

In [ ]:
resample = y.resample("M")

In [ ]:
resample.mean()

Il est aussi possible d'augmenter la fréquence, en donnant optionnellement la méthode de remplissage

In [ ]:
y.asfreq('H', method='ffill')

Les décalages sont aussi faciles à obtenir

In [ ]:
y

In [ ]:
y.shift(1)

In [ ]:
y.shift(-1)

### Fenêtres Glissantes

* Quand on travaille avec des séries temporelles, souvent on a besoin de "glisser" une fenêtre de temps
* Utiliser l'objet **Rolling** 

In [ ]:
ts = pd.Series(np.random.randn(1000), index=pd.date_range('1/1/2000', 
                                                          periods=1000))
ts = ts.cumsum()

In [ ]:
rolling = ts.rolling(window=60)
rolling

In [ ]:
rolling.mean()

### Exercice

* Créer une colonnedatetime 'date' à partir de NFS_1974.csv
  * styr: Survey year
  * stmth: Survey month
  * logday: Day in the log (assume logdays are actual days)
* *Astuce*: deux façons de le faire
  * utiliser l'option `parse_dates` de `read_csv`
  * créer la colonne date après la lecture du DataFrame

In [ ]:
# [Solution here]

In [ ]:
%load solutions/load_nfs_datetime.py


## Fusion et Jointure de DataFrames

In [ ]:
# Cette lecture est un peu lente à cause du "parsing" des colonnes

transit = pd.read_csv("./data/AIS/transit_segments.csv", 
                      parse_dates=['st_time', 'end_time'],
                      infer_datetime_format=True)
vessels = pd.read_csv("./data/AIS/vessel_information.csv")

* Les informations issues des bases de données sont souvent normalisées
* C'est à dire, avec des informations redondantes disposées sur plusieurs tableaux (ex : clés étrangères)
* Pour travailler on doit fusionner (*merge*) ou faire la jointure (*join*) des tableaux

In [ ]:
vessels.head()

In [ ]:
transit.head()

* On voit que plusieurs navires ont effectué plusieurs voyages sur les mêmes trajets
* Associer les déplacements aux noms des vaisseaux.

On trouve quelle est la colonne commune à ces deux tableaux

In [ ]:
vessels.columns.intersection(transit.columns)

### Merging

L'opération `merge` va associer les tableaux à partir des colonnes communes (sauf si on demande explicitement d'utiliser autres colonnes)

In [ ]:
transit.merge(vessels).head()

**attention**, Lors d'un merge sur les colonnes, les indices sont ignorés

In [ ]:
A = pd.DataFrame(np.random.randn(25, 2), 
                 index=pd.date_range('1/1/2015', periods=25))
A[2] = np.repeat(list('abcde'), 5)
A

In [ ]:
B = pd.DataFrame(np.random.randn(5, 2))
B[2] = list('abcde')
B

In [ ]:
A.merge(B, on=2)

### Jointures

* Les jointures sont comme les merge, mais travaillent à partir des index


In [ ]:
transit.set_index('mmsi', inplace=True)
vessels.set_index('mmsi', inplace=True)

In [ ]:
transit.join(vessels).head()

## Concatenation

* Autre opération commune est la concaténation de lignes ou de colonnes à un DataFrame existant
* On peut utilise la fonction `concat`
* Ici, on accédera à deux rapports de l'évolution du Ebola. On veut les associer pour pouvoir faire des analyses pousées sur les données.

In [ ]:
df1 = pd.read_csv('./data/ebola/guinea_data/2014-08-04.csv', 
                  index_col=['Date', 'Description'])
df2 = pd.read_csv('./data/ebola/guinea_data/2014-08-26.csv',
                 index_col=['Date', 'Description'])

In [ ]:
df1.shape

In [ ]:
df2.shape

In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
df1.index.is_unique

In [ ]:
df2.index.is_unique

Dans ce cas, il suffit de concatener les lignes

In [ ]:
df = pd.concat((df1, df2), axis=0)
df.shape

## Données textuelles

* Tout comme les données catégoriques et datetime vus précédemment, les strings ont un opérateur `str` avec des opérations utiles

In [ ]:
vessels.type

* Compter le nombre de séparateur (/) dans les types de navires.

In [ ]:
vessels.type.str.count('/').max()

* Eclater ces champs avec le séparateur '/' et les retourner (avec un remplissage `None` si nécessaire)

In [ ]:
vessels.type.str.split('/', expand=True)